After training **EfficientNetB0** models in this [notebook](https://www.kaggle.com/code/umongsain/efficientnetb0-tpu-pytorch), we load the models and predict on the test data.

In [ ]:
! pip install -q timm

In [ ]:
import glob
import io

import numpy as np
import pandas as pd

import tensorflow as tf
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as T

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

In [ ]:
def tfrecords_to_dataframe(fp, test=False):

    def parse(pb, test=False):
        d = {
            "id": tf.io.FixedLenFeature([], tf.string),
            "image": tf.io.FixedLenFeature([], tf.string),
        }
        if not test:
            d["class"] = tf.io.FixedLenFeature([], tf.int64)
        return tf.io.parse_single_example(pb, d)

    df = {"id": [], "img": []}
    if not test:
        df["lab"] = []
    for sample in tf.data.TFRecordDataset(glob.glob(fp)).map(
        lambda pb: parse(pb, test)
    ):
        df["id"].append(sample["id"].numpy().decode("utf-8"))
        df["img"].append(sample["image"].numpy())
        if not test:
            df["lab"].append(sample["class"].numpy())
    return pd.DataFrame(df)

In [ ]:
 test_df = tfrecords_to_dataframe(
     "../input/tpu-getting-started/tfrecords-jpeg-224x224/test/*.tfrec",
     test=True
 )

In [ ]:
class PetalDataset(Dataset):
    
    def __init__(self, df, test=False):
        self.df = df
        self.test = test
        self.transform = T.ToTensor()
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        img = Image.open(io.BytesIO(item.img))
        img = self.transform(img)
        
        if self.test:
            return img
        
        label = item.lab
        return img, label

In [ ]:
test_ds = PetalDataset(test_df, test=True)
test_loader = DataLoader(test_ds, batch_size=8)

In [ ]:
model_paths = glob.glob('/kaggle/input/efficientnetb0-tpu-pytorch/*.pt')
model = timm.create_model('efficientnet_b0', pretrained=False, num_classes=104)

In [ ]:
@torch.no_grad()
def infer(model, test_loader, device):
    preds = []
    for data in  tqdm(test_loader):
        data = data.to(device)
        logits = model(data)
        proba = logits.softmax(axis=-1)
        preds.append(proba.cpu().numpy())
    return np.concatenate(preds, axis=0)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

all_preds = []
for model_fp in model_paths:
    model.load_state_dict(torch.load(model_fp))
    all_preds.append(infer(model, test_loader, device))

In [ ]:
submit_df = pd.DataFrame(
    dict(
        id = test_df['id'],
        label = np.mean(all_preds, axis=0).argmax(axis=-1),
    )
)


In [ ]:
submit_df.to_csv('submission.csv', index=False)